<a href="https://colab.research.google.com/github/LoniQin/lifelong-ml/blob/main/Chatbot_with_Qwen2_and_MESOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot with Qwen2 and MESOP

## Intruduction

In this notebook, we will build a chatbot using Qwen2, a language model from the Hugging Face Transformers library, integrated with Gradio for a user-friendly interface. Qwen2 is a powerful model capable of generating human-like text responses based on input messages. MESOP makes it possible to write HTML page using Python and show HTML page in Google Colab.


## Setup and Installation

In [1]:
!pip install -q accelerate mesop

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 67.8 MB/s eta 0:00:00


In [2]:
import transformers
from tqdm.notebook import tqdm
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import numpy as np
import pickle
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)
print(f"Version: {transformers.__version__}")
import os

Version: 4.41.2


Load Qwen2 Model: Utilize Hugging Face’s Transformers library to load the Qwen2 model. Qwen2 is pretrained on large datasets, enabling it to generate coherent and context-aware responses.

In [3]:
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-7B-Instruct",
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



This code snippet generates a response from a large language model based on a given prompt. First, it sets up a conversation context where the system introduces itself and prompts the user with a specified message. The tokenizer.apply_chat_template function prepares the messages for input into the model, ensuring they are formatted appropriately for dialogue generation. The model_inputs are then tokenized and converted into PyTorch tensors for compatibility with the specified device. Using model.generate, the model generates a response, ensuring that it produces up to 512 new tokens beyond the input tokens. Finally, the generated response is decoded using the tokenizer to remove special tokens, and the resulting response is printed to the console. This process allows for the interactive generation of text-based responses from a language model based on user prompts, facilitating natural and context-aware interactions.

In [4]:
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(f"response:{response}")

response:A large language model is a type of artificial intelligence (AI) that has been trained on vast amounts of text data, enabling it to understand and generate human-like language. These models are incredibly complex and typically consist of billions or even trillions of parameters, which give them the ability to capture intricate patterns and nuances in language.

The training process involves feeding the model a massive dataset, often from the internet, so that it can learn the relationships between words, sentences, and paragraphs. Once trained, these models can be used for various tasks, such as text completion, translation, summarization, question answering, and even generating creative content like poetry or stories.

Some popular examples of large language models include GPT-3 ( by OpenAI, BERT ( by Google, and T5 by Google. These models have revolutionized the field of natural language processing (NLP) and have applications across numerous industries, from customer service

## Launch MESOP

In [5]:
import mesop as me
import mesop.labs as mel

me.colab_run()


Running server on: http://localhost:32123
 * Serving Flask app 'mesop.server.server'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (::)
 * Running on http://[::1]:32123
 * Running on http://[::1]:32123
INFO:werkzeug:Press CTRL+C to quit


In [6]:
@me.page(path="/hello_world")
def hello_world():
  me.text("Hello World!")

In [7]:
me.colab_show(path="/hello_world", height=100)

<IPython.core.display.Javascript object>

## Building the Chatbot

Implement a Python function that interacts with Qwen2. This function processes user messages, maintains conversation history, and generates responses using Qwen2. Also implement a chat interface with Python.

In [20]:
@me.page(path="/chat")
def chat():
  mel.chat(transform, title="Qwen 2", bot_user="Qwen 2")

def transform(prompt: str, history: list[mel.ChatMessage]) -> str:
    messages = []
    for item in history:
      if item.role == "user":
        messages.append({"role": "user", "content": item.content})
      else:
        messages.append({"role": "system", "content": item.content})
    messages.append({"role": "user", "content": prompt})
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [21]:
me.colab_show(path="/chat")

<IPython.core.display.Javascript object>